In [47]:
from tqdm import tqdm
import numpy as np
import os
import dask

In [31]:
import importlib

In [7]:
from coppafish.pipeline.run import initialize_nb
from coppafish.utils import nd2

In [3]:
nb = initialize_nb('/home/mathieubo/AgeingProject/WT50_28mo.ini', True)

/home/mathieubo/iss_dev/coppafish/coppafish/setup/notebook.py:279: UserWarning: Notebook file not found, creating a new notebook.
  warnings.warn("Notebook file not found, creating a new notebook.")
/home/mathieubo/iss_dev/coppafish/coppafish/pipeline/basic_info.py:66: UserWarning: Anchor file given and anchor channel specified.
Will use anchor round, channel 18 as reference
  warnings.warn(f"Anchor file given and anchor channel specified."


Image file contain a single tile, changing metadata format


Reading XY metadata: 58it [00:02, 26.51it/s]
/home/mathieubo/anaconda3/envs/coppafish_dev/lib/python3.9/site-packages/numpy/lib/type_check.py:300: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return imag(x) == 0


Notebook saved: took 0.01612257957458496 seconds


In [27]:
from coppafish.utils.raw import load_dask

In [28]:
da = load_dask(nb.file_names, nb.basic_info, 0)

Loading tiles in dask array: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 58/58 [00:42<00:00,  1.38it/s]


In [4]:
n_tiles = 58
n_lasers = 7
r = 0

In [5]:
nbp_file = nb.file_names

In [25]:
round_laser_dask_array = []

#Get all files of a given round
round_files = nbp_file.round[r*n_tiles*n_lasers:(r+1)*n_tiles*n_lasers]

for t in tqdm(range(n_tiles), desc='Loading tiles in dask array'):
    #Get all the files of a given tiles (should be 7)
    tile_files = round_files[t*n_lasers: (t+1)*n_lasers]
    tile_dask_array = []
    
    for f in tile_files:
        laser_file = os.path.join(nbp_file.input_dir, f + '.nd2')
        tile_dask_array.append(nd2.load(laser_file))
    tile_da = dask.array.concatenate(tile_dask_array, axis=-1)  # concatenate on the laser axis
    tile_da = dask.array.swapaxes(tile_da, -1, 0)  # we need 'channel', 'z', 'y','x'

    round_laser_dask_array.append(tile_da)

Loading tiles in dask array: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 58/58 [00:41<00:00,  1.38it/s]


In [35]:
rda = dask.array.stack(round_laser_dask_array, axis=0)

In [37]:
from coppafish.utils.raw import load_image

In [39]:
a = load_image(nb.file_names, nb.basic_info, t=5, c=0, round_dask_array=rda, r=0)

In [42]:
import napari

In [43]:
viewre = napari.Viewer()

In [44]:
viewre.add_image(a)

<Image layer 'a' at 0x7f0e8c74d820>

In [50]:
use_channels = nb.basic_info.use_channels
use_z = nb.basic_info.use_z
nbp_basic = nb.basic_info

In [51]:
image_max = np.zeros((len(use_channels), len(use_z)))
for i in range(len(use_channels)):
    image_max[i, :] = np.max(np.max(load_image(nbp_file, nbp_basic, t, use_channels[i],
                                                         rda, r, use_z), axis=0), axis=0)
max_channel = use_channels[np.max(image_max, axis=1).argmax()]
max_z = use_z[np.max(image_max, axis=0).argmax()]

In [53]:
max_channel

23

In [54]:
nb.file_names.raw_extension

'jobs'